This notebook is a step by step guide about how to train a deep neural network (DNN) in the DeepDeconv framework.

In [1]:
## Set up the sys.path in order to be able to import our modules
import os
import sys
module_path = os.path.abspath(os.path.join('../python'))
if module_path not in sys.path:
    sys.path.append(module_path)
os.environ["CUDA_VISIBLE_DEVICES"]="1" #"0" for the 1st GPU or "1" to use the 2nd

In [2]:
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k

###################################
# TensorFlow wizardry
config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
# This line is optional, don't add it unless you really need to set a limit on the memory available for your process
# For instance, if you want to train 2 DNNs on the same GPU without one overlapping the memory needed by the other
# Change the value to set the percentage of memory allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.75 

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Now you can create/load your DNN

Using TensorFlow backend.


We load the class of the network. This class must inherit from the DeepNet superclass. The method build_model has to be redefined in the child class with the wanted architecture. In our work, we use the network defined in deconvnNet. 

In [3]:
from DeepDeconv.deepnetFCS.DeconvNet_custom import DeconvNet

# 1) The network 
The purpose of this next code is to create the Dense XCeption U-Net with all the necessary parameters (number of scales,layers, growth rate, weighting factor, etc.)

In [1]:
nb_scales = 4 #4
growth_rate = 12 #12
nb_layers_per_block = [4,5,6,7]#[4,5,6,7]
activation_function= 'relu' #'relu'
gamma=0  #weighting factor for the loss function
shape_constraint=True # or can be shearlet = True or if nothing is specified, both shape_constraint and shearlet are False
atrou=False
resNet=False
layer_string='layer{0}'.format(nb_layers_per_block[0])

if shape_constraint:
    shape='shape_constraint
elif shearlet:
    shape='shearlet'
else:
    shape='noshape'


for k in range(1,len(nb_layers_per_block)):
    layer_string+='x{0}'.format(nb_layers_per_block[k])
network_name='ShapeNet2D_claire_sc{0}_{1}_{2}_growthRate{3}_{4}_gamma{5}'.format(nb_scales,layer_string,
                                                                              activation_function,growth_rate,shape,gamma)
if resNet:
    network_name+='_resNet'

dnn = DeconvNet(network_name = network_name, img_rows = 96, img_cols = 96, model_file='', verbose=True,
                nb_scales=nb_scales, growth_rate=growth_rate, nb_layers_per_block=nb_layers_per_block, 
                activation_function=activation_function,resNet=resNet,atrou=atrou,gamma=gamma,
                shape_constraint=shape_constraint)



NameError: name 'DeconvNet' is not defined

In [ ]:
#Input the directory containing the fits file
data_directory = '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/'
write_path="/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/"

#Retrieves the list of all the files
import glob

gal_files = glob.glob(data_directory+'image-*-multihdu.fits')
gal_files.sort()
print(gal_files)

#if using windows
win_files = glob.glob(write_path+'window/'+'Gaussian*')
win_files.sort()
print(win_files)


SNR = [20,100]#Range of SNR simulated
noiseless_img_hdu = 0
psf_hdu = 1
targets_hdu = 2
deconv_mode = 'TIKHONOV'

['/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-0-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-1-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-10-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-11-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-12-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-13-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-14-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-15-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-16-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-17-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-18-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-19-multihdu.fits', '/data/DeepDeconv/data/vsc_euclidpsfs/reshuffle/image-shfl-2-mult

In [ ]:
#Train with the image-000-0.fits as validation and all the other files as training set
#dnn.train_generator(gal_files[1:], gal_files[0], epochs=20, batch_size=32,
#                        nb_img_per_file=10000, validation_set_size=10000,
#                        noise_std=None, SNR=SNR, model_file='',
#                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu, psf_hdu=psf_hdu,
#                        image_dim=96, image_per_row=100,
#                        deconv_mode=deconv_mode)
#Here the number of epochs is set to 2, should be on the order of 20 at the end

dnn.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='', nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],
                        win_filename=win_files[2:],win_hdu=0,mom_hdu=1)

Model will be saved at /home/cben-ali/Programs/deep-deconv/tutorials/ShapeNet2D_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs.hdf5
Memory usage for the model + one batch (GB): 2.809000
Not using any previous monitored value for checkpoint
Instructions for updating:
Use tf.cast instead.


/home/cben-ali/Programs/deep-deconv/python/DeepDeconv/deepnetFCS/DeepNet.py:363: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=10, validation_data=([array([[..., verbose=1, callbacks=[<DeepDeco..., initial_epoch=0, steps_per_epoch=5947)`
  initial_epoch=initial_epoch)


Epoch 1/10
1881/5947 [========>.....................] - ETA: 1:23:27 - loss: 1.2312 - weighted_mean_squared_error: 1.2312 - weighted_shape_metric: 0.0000e+00

# 2) Alexis net : Restart with gamma=0
Used to check if training without shape constraint is equivalent to training with shape constraint and gamma=0.

In [ ]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs.hdf5",
             shape_constraint=True,gamma=0) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

In [ ]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_test_ref_restart.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

In [ ]:
del dnn2

# 3) Alexis net : Restart with gamma=0.1

In [ ]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs.hdf5",
             shape_constraint=True,gamma=0.1) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

In [ ]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.1_restart.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

In [ ]:
del dnn2

# 4) Alexis net : Restart with gamma=0.01

In [ ]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs.hdf5",
             shape_constraint=True,gamma=0.01) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

In [ ]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.01_restart.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

In [ ]:
del dnn2

# 5) Alexis net : Restart with gamma=0.001

In [ ]:
from DeepDeconv.deepnetFCS.DeepNet import DeepNet
dnn2=DeepNet(model_file="ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_preshape_10epochs.hdf5",
             shape_constraint=True,gamma=0.001) 
print(dnn2.model.inputs)
print(dnn2.model.input_names)

In [ ]:
dnn2.train_generator(gal_files[2:], gal_files[1], epochs=10, batch_size=32,
                        model_file='ShapeNet2D_FCS_claire_sc4_layer4x5x6x7_relu_growthRate12_gamma0.001_restart.hdf5', 
                        nb_img_per_file=10000, 
                        validation_set_size=10000, noise_std=None, SNR=SNR, 
                        noiseless_img_hdu=noiseless_img_hdu, targets_hdu=targets_hdu,
                        psf_hdu=psf_hdu, image_dim=96, image_per_row=100,
                        deconv_mode=deconv_mode, win_validation_filename=win_files[1],win_filename=win_files[2:],
                        win_hdu=0,mom_hdu=1)#,keep_best_loss=False by default

# Appendix for GPU options

In [ ]:
## extra imports to set GPU options
#import tensorflow as tf
#from keras import backend as k

###################################
# TensorFlow wizardry
#config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
#config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
# This line is optional, don't add it unless you really need to set a limit on the memory available for your process
# For instance, if you want to train 2 DNNs on the same GPU without one overlapping the memory needed by the other
# Change the value to set the percentage of memory allocated
#config.gpu_options.per_process_gpu_memory_fraction = 0.5 

# Create a session with the above options specified.
#k.tensorflow_backend.set_session(tf.Session(config=config))

# Now you can create/load your DNN

Same with multiple GPUs, Tensorflow will pre-allocate the whole memory of all the GPUs.
Use the following to prevent it (only when your station has several GPUs like the SAPPCN63):

In [ ]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0" #"0" for the 1st GPU or "1" to use the 2nd